# Scryfall Price Trend Harvester

This program extracts magic the gathering (mtg) card information and prices from the Scryfall rest-API.

The idea is to get the bulk data in order to be able to store the all cards information (more info than the API will return in a normal query). If you need help understanding the data structure or you would like to query only for specific info, then look the Jupyter Notebook call "mtg_cards_query.ipynb". In there is an example of how to do that.

In [1]:
#Import libreries
import requests
import pandas as pd
from datetime import datetime

import sqlite3

### Setup

In [2]:
DEBUG = False

#Current date
now = datetime.now()
date_time = str(now.strftime("%Y%m%d"))

#pd.set_option('display.max_colwidth', None)

### Getting Bulk Data

Downloading the full card json that is generated every day.
More info about it in: https://scryfall.com/docs/api/bulk-data

In [3]:
api_response = requests.get('https://api.scryfall.com/bulk-data/default-cards')
print(api_response.status_code) #should be 200=ok

200


In [4]:
#Translate the response object's content from bytes to dictionary object so 
#it can be easily manipulate it
api_json = api_response.json() #converts from bytes to dictionary

#[Debug]
if DEBUG:
    print(api_json['download_uri'])

In [5]:
#Get cards information from bulk data in the responsed url.
bulk_url_json = requests.get(api_json['download_uri'], allow_redirects=True)

##Save data as json file in disk
#file_size = open('default_cards.json', 'wb').write(bulk_url_json.content)
#[Debug]
#if DEBUG:
    #print("Json downloaded correctly. File size: " + str(file_size))

##Save data to pandas
#Load json data to a pandas Data frame.
url_json = bulk_url_json.json() #converts from bytes to dictionary
cards_bulk_df = pd.DataFrame(url_json)

#[Debug]
if DEBUG:
    cards_bulk_df.to_csv('initial_bulk.csv', sep=";", index=False) 

#[Debug]
if DEBUG:
    #All columns types and info
    cards_bulk_df.info(verbose=True)
    #Summary of the first 3 rows
    print(cards_bulk_df.head(3))

#### Prices column 
(a dictionary of prices)

Ejemplo: {'usd': '76.45', 'usd_foil': '81.20', 'usd_etched': None, 'eur': '38.64', 'eur_foil': '43.93', 'tix': '1.12'}

In [6]:
#[Debug]
if DEBUG:
    print(cards_bulk_df['prices'].head(3))

### Save Data to local DB

First we need to filter the data that we don't want and keep the rest.

In my case is not only about the columns but also about the card editions. I'm an old school Magic collector so I love the old frames/border cards. I'll limit the db to that.

| id | name | mana_cost | cmc | power | toughness | reserved | foil | nonfoil | set_id | set_name | rarity | frame | date_time | small | normal | usd | usd_foil | usd_etched | eur | eur_foil |
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| 6c900f34-1bd2-43c7-be33-cf5cc02a62ea | Replenish | {3}{W} | 4.0 |  |  | True | False | True | 632741a4-411d-4110-b507-5a5cfdd52ef2 | World Championship Decks 2000 | rare | 1997 | 20220417 | https://c1.scryfall.com/file/scryfall-cards/small/front/6/c/6c900f34-1bd2-43c7-be33-cf5cc02a62ea.jpg?1562767042 | https://c1.scryfall.com/file/scryfall-cards/normal/front/6/c/6c900f34-1bd2-43c7-be33-cf5cc02a62ea.jpg?1562767042 | 40.15 |  |  | 25.00 | | 
| 7fd2fe13-bbc0-42b7-bc42-3b51910ce118 | Replenish | {3}{W} | 4.0 |  |  | True | True | True | 44f17b37-dcf8-4239-baab-1efc00cd3480 | Urza's Destiny | rare | 1997 | 20220417 | https://c1.scryfall.com/file/scryfall-cards/small/front/7/f/7fd2fe13-bbc0-42b7-bc42-3b51910ce118.jpg?1562444251 | https://c1.scryfall.com/file/scryfall-cards/normal/front/7/f/7fd2fe13-bbc0-42b7-bc42-3b51910ce118.jpg?1562444251 | 114.42 | 925.99 |  | 74.24 | 571.50 |

In [7]:
columns_list = ['id', 'name', 'image_uris', 'mana_cost', 'cmc', 'power',
                'toughness', 'reserved', 'foil', 'nonfoil', 'set_id',
                'set_name', 'rarity', 'frame', 'prices', 'date_time']

#General Cleaning
#---------------------------------------------------
#Discard any card that's not in english
cards_df_flt = cards_bulk_df.loc[cards_bulk_df["lang"] == "en"]
#Discard the cards without image
cards_df_flt = cards_df_flt.dropna(subset = ['image_uris'])
#Adds date_time to the Data frame
cards_df_flt['date_time'] = date_time
#Keep only necessary columns
cards_df_flt = cards_df_flt[columns_list]

#[Debug]
if DEBUG:
    #All columns types and info
    cards_df_flt.info(verbose=True)

Now we split the columns with dictionary-like objects to obtain a "plain table" and store it on the DB.

In [8]:
#Split dictonary column for cards images
cards_df_flt = cards_df_flt.join(cards_df_flt.image_uris.apply(pd.Series), how='left')

#Split dictonary column for cards prices
cards_df_flt = cards_df_flt.join(cards_df_flt.prices.apply(pd.Series), how='left')

#Drop innecesary columns after de split
cards_df_flt.drop(["image_uris", "prices", "large", "png", "art_crop", "border_crop", "tix"], axis=1, inplace=True)

#### Connect to the DB

In [9]:
#Generates (or creates if it doesn't exist) the db connection.
conn = sqlite3.connect("sqlite_db/mtg_cards.db")

#### Load DataFrame to table

In [10]:
#Load pandas Data frame into scryfall_cards table.
cards_df_flt.to_sql(name="scryfall_cards", con=conn, if_exists="append", index=False)

63829

#### Testing result

In [13]:
#SQL query for testing
sql_result = conn.execute(
    """ SELECT
            id,
            name,
            set_name,
            usd,
            usd_foil
        FROM scryfall_cards
        WHERE
            name = 'Replenish'
            AND
            date_time = '20220417'
        ORDER BY id ASC
;""")

#Obtains the table column names because sqlite query returns only the data.
colums_names = [column[0] for column in sql_result.description]
#Creates a pandas dataframe qith the query data and the column names.
sql_df= pd.DataFrame.from_records(data = sql_result.fetchall(), columns = colums_names)

print(sql_df.head(5))

                                     id       name  \
0  6c900f34-1bd2-43c7-be33-cf5cc02a62ea  Replenish   
1  7fd2fe13-bbc0-42b7-bc42-3b51910ce118  Replenish   

                        set_name     usd usd_foil  
0  World Championship Decks 2000   40.15     None  
1                 Urza's Destiny  114.42   925.99  


#### Close connection to DB

In [14]:
#Close connection
conn.close()